In [13]:
import pandas as pd
import numpy as np
#pd.read_csv('application_train.csv')

## Model Implementation (Each function computes the current gradient)

In [101]:
# Logistic Regression Gradient
def logistic(X, Y, B):
    half =  np.exp(X @ B)
    p = half / (1 + half)
    gradient = -1 * (Y - p).T @ X
    
    return gradient

# Logistic Regression with Lasso Penalty
def logistic_lasso(X, Y, B, lamb):
    half =  np.exp(X @ B)
    p = half / (1 + half)
    partial_gradient = -1 * (Y - p).T @ X
    
    # Add on +/- lambda * B to the gradient
    lamb_beta = np.nan_to_num(lamb * -1 * (B / (B * -1)))
    gradient = partial_gradient + lamb_beta
    
    return gradient

def predict_prob(X, B):
    return 1 / (1 + np.exp(-1 * X @ B))


def SVC(X, Y, B, lamb):
    # REQUIRES CLASS LABELS 1 / -1
    Y = (Y - 1) + Y
    
    # I know this looks crazy but trust it works :)
    mask = (1 - (Y * (X @ B))) <= 0

    if_not_0_replace_w = B - (lamb * (X.T * Y).T)
    return (B * mask.sum() + if_not_0_replace_w[~mask].sum(axis=0)) / len(Y)

In [75]:
Y = np.array([1, -1, 1, -1])
X = np.array([[1, 2], [1, 3], [1, 4], [1, 15]])
B = np.array([5, 3])

if_not_0_replace_w = B - (X.T * Y).T

mask = (1 - (Y * (X @ B))) <= 0
B * mask.sum()
(B * mask.sum() + if_not_0_replace_w[~mask].sum(axis=0)) / len(Y)

array([5.5, 7.5])

In [5]:
heart = pd.read_csv('https://www.dropbox.com/s/jpnyx41u7wpa41m/heart_attack_clean.csv?dl=1')
heart_X = heart.drop(columns=['output'])
heart_X['B0'] = 1
heart_X = heart_X[['B0'] + list(heart_X.drop(columns='B0').columns)]
heart_X = heart_X.to_numpy()
heart_Y = heart['output'].to_numpy()

In [108]:
heart_SVC_B = gradient_descent(SVC, heart_X, heart_Y, 10)
p_logistic = predict_prob(heart_X, heart_SVC_B)
compute_metrics(p_logistic, heart_Y)

Eta: 0.1; Iterations: 75000
Eta: 0.01; Iterations: 75000
Gradient converged w/ 1682 iterations and eta = 0.001
Accuracy: 0.791; Precision: 0.799; Recall: 0.815; f1: 0.807


In [100]:
import sklearn
clf = sklearn.svm.SVC(kernel='linear')
clf.fit(heart_X, heart_Y)
clf.score(heart_X, heart_Y)

0.8021978021978022

## Evaluation Metrics

In [6]:
def get_accuracy(p, Y):
    return sum((p > .5) == Y) / len(Y)

def get_precision(p, Y):
    # precision = TP / (TP + FP)
    TP = np.where(Y, ((p > .5) == Y), False).sum()
    FP = np.where(Y == 0, (p > .5), False).sum()
    return TP / (TP + FP)
   
def get_recall(p, Y):
    # recall = TP / (TP + FN)
    TP = np.where(Y, ((p > .5) == Y), False).sum()
    FN = np.where(Y, ((p > .5) != Y), False).sum()
    return TP / (TP + FN)

def get_f1(p, Y):
    precision = get_precision(p, Y)
    recall = get_recall(p, Y)
    return 2 / ((1 / precision) + (1 / recall))
    
# Compute Accuracy, precision, recall, and f1
def compute_metrics(p, Y):
    # precision = TP / (TP + FP)
    # recall = TP / (TP + FN)
    accuracy = sum((p > .5) == Y) / len(Y)
    
    TP = np.where(Y, ((p > .5) == Y), False).sum()
    FP = np.where(Y == 0, (p > .5), False).sum()
    precision = TP / (TP + FP)
    
    FN = np.where(Y, ((p > .5) != Y), False).sum()
    recall = TP / (TP + FN)
    
    f1 = 2 / ((1 / precision) + (1 / recall))
    print(f"Accuracy: {round(accuracy, 3)}; Precision: {round(precision, 3)}; " + \
           f"Recall: {round(recall, 3)}; f1: {round(f1, 3)}")

## Gradient Descent Implementation (Pass in a regression function, X, Y, and any optional arguments)

In [11]:
def gradient_descent(reg_func, X, Y, *reg_func_args, initial_B=None, max_iterations=75000, tol = .00001,
                     etas=[.1, .01, .001, .0001, .00001, .000001], err=False):
    if not err:
        np.seterr(all="ignore")
    else:
        np.seterr(all="warn")
    
    for eta in etas:
        # reset
        iterations = 0
        if initial_B is not None:
            B = initial_B
        else:
            B = np.zeros(len(X[0]))
        gradient = np.zeros(len(X[0]))
        while iterations < max_iterations and np.isinf(B).sum() == 0 and \
              (iterations == 0 or (eta * (gradient ** 2)).sum() > tol):
            # calls the regression function
            gradient = reg_func(X, Y, B, *reg_func_args)
            B = B - (eta * gradient)
            iterations += 1

        if iterations < max_iterations and np.isinf(B).sum() == 0 and np.isnan(B).sum() == 0:
            print(f'Gradient converged w/ {iterations} iterations and eta = {eta}')
            np.seterr(all="warn")
            return B
        print(f'Eta: {eta}; Iterations: {iterations}')
    print('GRADIENT DID NOT CONVERGE. RESULTS ARE BAD')
    np.seterr(all="warn")
    return B

# The code below uses the 'Adagrad' gradient descent optimization algorithm to adapt the 
# learning rate for each dimension. There are other versions of this that may be more effective
# Directions as to how this works as well as other ideas: 
# https://ruder.io/optimizing-gradient-descent/index.html#momentum
def adaptive_gradient_descent(reg_func, X, Y, *reg_func_args, initial_B=None, max_iterations=100000, 
                              tol = .001, etas=[.1], err=False):
    if not err:
        np.seterr(all="ignore")
    else:
        np.seterr(all="warn")
    
    for eta in etas:
        # reset
        iterations = 0
        if initial_B is not None:
            B = initial_B
        else:
            B = np.zeros(len(X[0]))
        gradient = np.zeros(len(X[0]))
        SS_past_gradients = np.zeros(len(X[0]))
        while iterations < max_iterations and np.isinf(B).sum() == 0 and \
              (iterations == 0 or (eta * (gradient ** 2)).sum() > tol):
            # calls the regression function
            gradient = reg_func(X, Y, B, *reg_func_args)
            
            # Where SS_past_gradients is sum of squares of past gradients
            SS_past_gradients += gradient ** 2
            #print(B)
            B = B - ((eta * gradient) / (np.sqrt(SS_past_gradients) + 1e-8))
            
            iterations += 1

        if iterations < max_iterations and np.isinf(B).sum() == 0 and np.isnan(B).sum() == 0:
            print(f'Gradient converged w/ {iterations} iterations and eta = {eta}')
            np.seterr(all="warn")
            return B
        print(f'Eta: {eta}; Iterations: {iterations}')
    print('GRADIENT DID NOT CONVERGE. RESULTS ARE BAD')
    np.seterr(all="warn")
    return B